## Import required packages

In [85]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dateutil
import matplotlib.pyplot as plt

## Load necessary datasets in form of pandas dataframe

In [86]:
## load residual measure data

# meeting minutes
df_measure_mm = pd.read_excel("../data/market_analysis_data/aggregate_measure_mm.xlsx", usecols=["ReleaseDate", "EndDate", "our_measure"])
df_measure_mm["EndDate"] = pd.to_datetime(df_measure_mm["EndDate"], format='%B/%d/%Y')
df_measure_mm["delay"] = (df_measure_mm["ReleaseDate"] - df_measure_mm["EndDate"]).dt.days

rolling_window = 24

df_measure_mm["SMA"] = df_measure_mm["our_measure"].rolling(rolling_window).mean()#df_measure_mm["our_measure"].shift(1) #df_measure_mm["our_measure"].rolling(rolling_window).mean()
print(df_measure_mm.head(5))
df_measure_mm = df_measure_mm.dropna()
print(df_measure_mm.head(5))

print(df_measure_mm["our_measure"].mean() + 2*df_measure_mm["our_measure"].std(), df_measure_mm["our_measure"].median())

threshold = df_measure_mm["our_measure"].mean() + 1*df_measure_mm["our_measure"].std()
threshold = 0.45

## load QQQ data
df_QQQ = pd.read_csv("../data/market_analysis_data/QQQ.csv")
df_QQQ["Date"] = pd.to_datetime(df_QQQ["Date"], format='%Y-%m-%d') 

'''
path_sp500 = "../data/market_analysis_data/SP500_HistoricalPrices.csv"
df_sp500 = pd.read_csv(path_sp500, usecols=["Date", " Close"])
df_sp500['Date'] = df_sp500['Date'].astype('datetime64[ns]')
df_QQQ = df_sp500.iloc[::-1]
'''


     EndDate ReleaseDate  our_measure  delay  SMA
0 1996-01-31  1996-03-29    -0.047170     58  NaN
1 1996-03-26  1996-05-24     0.220779     59  NaN
2 1996-05-21  1996-07-05     0.340909     45  NaN
3 1996-07-03  1996-08-23     0.274194     51  NaN
4 1996-08-20  1996-09-27     0.146667     38  NaN
      EndDate ReleaseDate  our_measure  delay       SMA
23 1998-12-22  1999-02-04     0.097222     44  0.182129
24 1999-02-03  1999-04-01     0.067416     57  0.186903
25 1999-03-30  1999-05-20     0.212500     51  0.186558
26 1999-05-18  1999-07-01     0.361111     44  0.187400
27 1999-06-30  1999-08-26     0.250000     57  0.186392
0.5043670337079758 -0.03571428571428571


'\npath_sp500 = "../data/market_analysis_data/SP500_HistoricalPrices.csv"\ndf_sp500 = pd.read_csv(path_sp500, usecols=["Date", " Close"])\ndf_sp500[\'Date\'] = df_sp500[\'Date\'].astype(\'datetime64[ns]\')\ndf_QQQ = df_sp500.iloc[::-1]\n'

In [87]:
df_merge = pd.merge(df_measure_mm, df_QQQ, left_on="ReleaseDate", right_on="Date", how="left")
df_merge["inter_meeting_period_ret"] = df_merge["Adj Close"].pct_change(1)*100
df_merge["inter_meeting_period_ret"] = df_merge["inter_meeting_period_ret"].shift(-1) 
df_merge = df_merge.dropna()
print(df_merge.shape)

print(df_merge.to_csv("temp.csv", index=False))

var_x = df_merge["our_measure"].tolist()

dates_list = [df_merge["Date"].tolist()[0]]
QQQ_BAH_list = [100.0]
QQQ_LSM_list = [100.0]
for i in range(0, len(var_x)):
    if i+1 == len(var_x):
        dates_list.append(df_measure_mm["ReleaseDate"].tolist()[-1])
    else:
        dates_list.append(df_merge["ReleaseDate"].tolist()[i+1])
    measure_value = df_merge["our_measure"].tolist()[i]
    rolling_avg = df_measure_mm["SMA"].tolist()[i]
    pct_change = df_merge["inter_meeting_period_ret"].tolist()[i]
    QQQ_BAH_list.append(QQQ_BAH_list[-1]*(1.0+(pct_change/100.0)))

    if measure_value <= 0.0: 
        QQQ_LSM_list.append(QQQ_LSM_list[-1]*(1.0+(pct_change/100.0)))
    else:
        QQQ_LSM_list.append(QQQ_LSM_list[-1]*(1.0-(pct_change/100.0)))

print(QQQ_BAH_list[-1], QQQ_LSM_list[-1])

fig = go.Figure()

fig.add_trace(go.Scatter(x=dates_list, y=QQQ_BAH_list,
    name="Buy and Hold"       # this sets its legend entry
))


fig.add_trace(go.Scatter(x=dates_list, y=QQQ_LSM_list,
    name="Our Strategy"
))



# Add figure title
fig.update_layout(
    title_text="Buy and Hold vs Our Strategy"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes title
fig.update_yaxes(title_text="Value of $100 Invested")

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    font=dict(
        size=25,
    ))

fig.show()



(189, 13)
None
568.9449643284522 228.53432044867884
